In [44]:
# Import necessary libraries
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA



In [45]:
df=pd.read_csv('heart.csv')

In [46]:
import pandas as pd
from scipy.stats import zscore

# Load the dataset
df = pd.read_csv('heart.csv')

# Identify the numerical columns
numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Calculate and display the Z-scores for each of the specified columns
z_scores = df[numeric_cols].apply(zscore)

# Display the Z-scores
print(z_scores.head())  # Shows the Z-scores for the first few rows


        age  trestbps      chol   thalach   oldpeak
0 -0.268437 -0.377636 -0.659332  0.821321 -0.060888
1 -0.158157  0.479107 -0.833861  0.255968  1.727137
2  1.716595  0.764688 -1.396233 -1.048692  1.301417
3  0.724079  0.936037 -0.833861  0.516900 -0.912329
4  0.834359  0.364875  0.930822 -1.874977  0.705408


In [47]:
# Step 3: Convert Categorical Columns to Numbers
# Apply One-Hot Encoding for specified categorical columns
df_encoded = pd.get_dummies(df, columns=['cp', 'restecg', 'slope', 'ca', 'thal'])

# Display the first few rows to see the encoding results
print(df_encoded.head())


   age  sex  trestbps  chol  fbs  thalach  exang  oldpeak  target  cp_0  ...  \
0   52    1       125   212    0      168      0      1.0       0  True  ...   
1   53    1       140   203    1      155      1      3.1       0  True  ...   
2   70    1       145   174    0      125      1      2.6       0  True  ...   
3   61    1       148   203    0      161      0      0.0       0  True  ...   
4   62    0       138   294    1      106      0      1.9       0  True  ...   

   slope_2   ca_0   ca_1   ca_2   ca_3   ca_4  thal_0  thal_1  thal_2  thal_3  
0     True  False  False   True  False  False   False   False   False    True  
1    False   True  False  False  False  False   False   False   False    True  
2    False   True  False  False  False  False   False   False   False    True  
3     True  False   True  False  False  False   False   False   False    True  
4    False  False  False  False   True  False   False   False    True   False  

[5 rows x 28 columns]


In [48]:
from sklearn.preprocessing import StandardScaler

# Step 4: Apply Scaling to Numerical Columns
scaler = StandardScaler()
numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Fit and transform the numeric columns and update them in the DataFrame
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Display the first few rows to see the scaled values
print(df[numeric_cols].head())


        age  trestbps      chol   thalach   oldpeak
0 -0.268437 -0.377636 -0.659332  0.821321 -0.060888
1 -0.158157  0.479107 -0.833861  0.255968  1.727137
2  1.716595  0.764688 -1.396233 -1.048692  1.301417
3  0.724079  0.936037 -0.833861  0.516900 -0.912329
4  0.834359  0.364875  0.930822 -1.874977  0.705408


In [49]:
# Step 5: Build Classification Models (SVM, Logistic Regression, Random Forest)
# Split data into features and target
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier()
}

# Train and evaluate each model
accuracy_results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy_results[name] = accuracy_score(y_test, y_pred)

# Print accuracy results for each model
print("Accuracy without PCA:", accuracy_results)

Accuracy without PCA: {'SVM': 0.8341463414634146, 'Logistic Regression': 0.7951219512195122, 'Random Forest': 0.9853658536585366}


In [50]:
# Step 6: Apply PCA and Retrain the Best Model (Random Forest)
# Apply PCA with a reasonable number of components (e.g., 2)
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Retrain Random Forest model on PCA-transformed data
best_model = RandomForestClassifier()
best_model.fit(X_train_pca, y_train)
y_pred_pca = best_model.predict(X_test_pca)
pca_accuracy = accuracy_score(y_test, y_pred_pca)

# Print accuracy after PCA
print("Accuracy with PCA (Random Forest):", pca_accuracy)

Accuracy with PCA (Random Forest): 0.9853658536585366
